# What about the LLMs?

LLMs are reputed to have revolutionised automatic language processing. Since the introduction of BERT-type models, all language processing applications have been based on LLMs, of varying degrees of sophistication and size. These models are trained on multiple tasks and are therefore capable of performing new tasks without learning, simply from a prompt. This is known as "zero-shot learning" because there is no learning phase as such. We are going to test these models on our classification task.

Huggingface is a Franco-American company that develops tools for building applications based on Deep Learning. In particular, it hosts the huggingface.co portal, which contains numerous Deep Learning models. These models can be used very easily thanks to the [Transformer] library (https://huggingface.co/docs/transformers/quicktour) developed by HuggingFace.

Using a transform model in zero-shot learning with HuggingFace is very simple: [see documentation](https://huggingface.co/tasks/zero-shot-classification)

However, you need to choose a suitable model from the list of models compatible with Zero-Shot classification. HuggingFace offers [numerous models](https://huggingface.co/models?pipeline_tag=zero-shot-classification). 

The classes proposed to the model must also provide sufficient semantic information for the model to understand them.

**Question**:

* Write a code to classify an example of text from an article in Le Monde using a model transformed using zero-shot learning with the HuggingFace library.
* choose a model and explain your choice
* choose a formulation for the classes to be predicted
* show that the model predicts a class for the text of the article (correct or incorrect, analyse the results)
* evaluate the performance of your model on 100 articles (a test set).
* note model sizes, processing times and classification results


Notes :
* make sure that you use the correct Tokenizer when using a model 
* start testing with a small number of articles and the first 100's of characters for faster experiments.

In [1]:
# Choisir modèle multilingue, essayer avec des modèles de différentes tailles.

## Installation des packages

In [10]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 47.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 68.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 53.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 96.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 98.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 83.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 94.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

In [11]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 67.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 91.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 99.7 MB/s eta 0:00:00


In [12]:
!pip install transformers

  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.4 MB/s eta 0:00:00


In [13]:
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.9 MB/s eta 0:00:00


In [42]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.2 MB/s eta 0:00:00


In [43]:
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00


## Importation des librairies

In [ ]:
import pandas as pd

In [14]:
import torch
import tensorflow as tf
from transformers import pipeline

2025-02-16 16:42:57.877567: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-16 16:42:57.895389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739724177.916862    3092 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739724177.923489    3092 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 16:42:57.946945: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [18]:
import time
from tqdm import tqdm

## Importation de la base de données et tirage de l'ensemble de test

In [3]:
 articles_df = pd.read_csv('LeMonde2003_9classes.csv.gz')

In [4]:
# Filter out the UNE class
articles_df = articles_df[articles_df['category'] != 'UNE']

In [5]:
sample = articles_df.sample(100)

In [6]:
sample

,text,category
9700,les autorités américaines ont demandé au corre...,INT
20933,sept millions de dollars par jour calculé par ...,INT
631,la concertation sur la réforme des retraites s...,FRA
13718,les souverainistes de droite sont dispersés da...,INT
14409,touché lors du premier match des all blacks fa...,SPO
...,...,...
14042,on retrouve presque inchangé le décor qui en 2...,ART
37,guerre ou paix la 75e cérémonie des academy aw...,ART
10746,maurad l'animateur vedette de nrj a vu son tal...,ENT
23181,la vie de surdoué n'est pas toujours aisée en ...,SPO


In [7]:
texts = sample.text

In [8]:
articles_df.category.unique()

array(['SPO', 'ART', 'FRA', 'SOC', 'INT', 'ENT'], dtype=object)

## Définition du modèle et des classes à prédire

In [9]:
# Noms des classes à prédire
candidate_labels = ['companies', 'international', 'arts', 'society', 'France', 'sports', 'font page articles']

# Dictionnaire de correspondance avec les valeurs de la base de données
corresp_dict = {
    'companies':'ENT',
    'international':'INT',
    'arts':'ART',
    'society':'SOC',
    'France':'FRA',
    'sports':'SPO',
    'font page articles':'UNE'
}

In [16]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Device set to use cpu


## Prédiction des classes sur l'ensemble de test et évaluation

In [19]:
predictions = []
scores = []

# Mesure du temps de traitement
start_time = time.time()
for text in tqdm(texts.to_list(), desc="Traitement", unit="texte"):
    result = classifier(text[:100], candidate_labels, multi_label=True)
    
    # Trouver l'indice du score le plus élevé
    max_score_index = result['scores'].index(max(result['scores']))

    # Sélectionner le label correspondant et son score
    selected_label = result['labels'][max_score_index]
    selected_score = result['scores'][max_score_index]

    # Ajouter les éléments aux listes
    predictions.append(selected_label)
    scores.append(selected_score)
    
# Temps de traitement
end_time = time.time()
processing_time = end_time - start_time

Traitement: 100%|██████████| 100/100 [02:54<00:00,  1.74s/texte]


NameError: name 'os' is not defined

In [23]:
# Afficher les résultats
print(f"Time processing: {processing_time:.4f} seconds")

Time processing: 174.0421 seconds


NameError: name 'model_size' is not defined

In [24]:
sample['prediction'] = predictions
sample['scores'] = scores

In [25]:
sample

,text,category,prediction,scores
9700,les autorités américaines ont demandé au corre...,INT,international,0.991142
20933,sept millions de dollars par jour calculé par ...,INT,international,0.981793
631,la concertation sur la réforme des retraites s...,FRA,international,0.901342
13718,les souverainistes de droite sont dispersés da...,INT,France,0.901902
14409,touché lors du premier match des all blacks fa...,SPO,sports,0.990045
...,...,...,...,...
14042,on retrouve presque inchangé le décor qui en 2...,ART,arts,0.602791
37,guerre ou paix la 75e cérémonie des academy aw...,ART,arts,0.981058
10746,maurad l'animateur vedette de nrj a vu son tal...,ENT,international,0.745293
23181,la vie de surdoué n'est pas toujours aisée en ...,SPO,sports,0.995362


In [26]:
# Remplacer les valeurs dans la colonne en utilisant le dictionnaire
sample['prediction'] = sample['prediction'].replace(corresp_dict)

In [27]:
# Calculer le nombre de classifications correctes
count_equal = (sample['category'] == sample['prediction']).sum()
total_lines = len(sample)
accuracy_percentage = (count_equal / total_lines) * 100

In [29]:
print(f"L'évaluation des performances du modèle montre que 'category' et 'prediction' sont égales dans {count_equal} cas sur {total_lines} lignes, soit un taux de précision de {accuracy_percentage:.2f}%")

L'évaluation des performances du modèle montre que 'category' et 'prediction' sont égales dans 41 cas sur 100 lignes, soit un taux de précision de 41.00%


Résultats faibles, essayer de ralonger le nombre de caractères considérés pour la prédiction + essayer d'autres modèles. 

## Expériences avec différents modèles et des échantillons plus longs

In [50]:
def evaluate_zero_shot_classification(model_name, text_length=100):
    """
    Effectue une classification zero-shot sur un échantillon des données.
    
    Paramètres:
    - df: DataFrame contenant une colonne 'text' et 'category'
    - model_name: Nom du modèle à utiliser
    - candidate_labels: Liste des labels possibles
    - sample_size: Nombre d'échantillons à utiliser (par défaut 100)
    - text_length: Nombre de caractères du texte pris en compte (par défaut 100)
    
    Retourne:
    - Un DataFrame avec les prédictions et scores
    - Le temps de traitement
    - L'accuracy du modèle
    """
    # Tirage de l'échantillon
    sample = articles_df.sample(100).copy()
    texts = sample['text']

    # Définition du modèle
    classifier = pipeline("zero-shot-classification", model=model_name)
    predictions, scores = [], []

    start_time = time.time()
    for text in tqdm(texts.to_list(), desc="Traitement", unit="text"):
        result = classifier(text[:text_length], candidate_labels, multi_label=True)
        
        max_score_index = result['scores'].index(max(result['scores']))
        selected_label = result['labels'][max_score_index]
        selected_score = result['scores'][max_score_index]
        
        predictions.append(selected_label)
        scores.append(selected_score)
    
    end_time = time.time()
    processing_time = end_time - start_time

    sample['prediction'] = predictions
    sample['scores'] = scores

    # Remplacement des labels selon un dictionnaire de correspondance
    sample['prediction'] = sample['prediction'].replace(corresp_dict)
    
    # Calcul de l'accuracy
    count_equal = (sample['category'] == sample['prediction']).sum()
    total_lines = len(sample)
    accuracy_percentage = (count_equal / total_lines) * 100
    
    print(f"Time processing: {processing_time:.4f} seconds")
    print(f"Accuracy: {accuracy_percentage:.2f}% ({count_equal}/{total_lines})")
    
    return sample, processing_time, accuracy_percentage

In [37]:
sample_results, time_taken, accuracy = evaluate_zero_shot_classification("facebook/bart-large-mnli", 300)

Device set to use cpu
Traitement: 100%|██████████| 100/100 [06:34<00:00,  3.95s/text]


Time processing: 394.8961 seconds
Accuracy: 49.00% (49/100)


In [52]:
# Liste des modèles à tester
AVAILABLE_MODELS = [
    "knowledgator/comprehend_it-base",
    "facebook/bart-large-mnli",
    "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", 
    "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", 
]

# Dictionnaire des tailles des modèles
MODEL_SIZES = {
    "knowledgator/comprehend_it-base": 0.47,
    "facebook/bart-large-mnli": 1.63,  # En Go
    "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli": 0.49,
    "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli": 0.49,
}

In [53]:
# Liste pour stocker les résultats
results = []

# Boucle sur les modèles
for model in AVAILABLE_MODELS:
    print(f"Testing model: {model}")
    sample_results, time_taken, accuracy = evaluate_zero_shot_classification(model, 100)

    # Ajouter les résultats dans la liste
    results.append({
        "Modèle": model,
        "Taille (Go)": MODEL_SIZES.get(model, "N/A"),
        "Temps (s)": round(time_taken, 2),
        "Accuracy (%)": round(accuracy, 2)
    })


Testing model: knowledgator/comprehend_it-base


Device set to use cpu
Traitement: 100%|██████████| 100/100 [02:27<00:00,  1.47s/text]


Time processing: 147.3231 seconds
Accuracy: 33.00% (33/100)
Testing model: facebook/bart-large-mnli


Device set to use cpu
Traitement: 100%|██████████| 100/100 [03:53<00:00,  2.34s/text]


Time processing: 233.9707 seconds
Accuracy: 29.00% (29/100)
Testing model: MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli


...

Time processing: 146.3255 seconds
Accuracy: 39.00% (39/100)
Testing model: MoritzLaurer/mDeBERTa-v3-base-mnli-xnli


Device set to use cpu
Traitement: 100%|██████████| 100/100 [02:27<00:00,  1.47s/text]


Time processing: 147.0754 seconds
Accuracy: 42.00% (42/100)
Testing model: joeddav/xlm-roberta-large-xnli


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast convertors: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'RealmTokenizer', 'Refor

In [54]:
# Convertir en DataFrame
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="Accuracy (%)", ascending=False)

# Afficher le tableau
print(df_results)

                                         Modèle  Taille (Go)  Temps (s)  \
3       MoritzLaurer/mDeBERTa-v3-base-mnli-xnli         0.49     147.08   
2  MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli         0.49     146.33   
0               knowledgator/comprehend_it-base         0.47     147.32   
1                      facebook/bart-large-mnli         1.63     233.97   

   Accuracy (%)  
3          42.0  
2          39.0  
0          33.0  
1          29.0  


Le modèle MoritzLaurer/mDeBERTa-v3-base-mnli-xnli a un taux de précision plus élevé que les autres modèles, alors que la taille et le temps d'exécution se placent parmi les plus faibles. 

## Evaluation avec plus de mots dans le contexte pour la prédiction

In [56]:
# Tirage de l'échantillon
sample = articles_df.sample(100).copy()
texts = sample['text']

# Définition du modèle
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
predictions, scores = [], []

start_time = time.time()
for text in tqdm(texts.to_list(), desc="Traitement", unit="text"):
    result = classifier(text[:500], candidate_labels, multi_label=True)
    
    max_score_index = result['scores'].index(max(result['scores']))
    selected_label = result['labels'][max_score_index]
    selected_score = result['scores'][max_score_index]
    
    predictions.append(selected_label)
    scores.append(selected_score)

end_time = time.time()
processing_time = end_time - start_time

sample['prediction'] = predictions
sample['scores'] = scores

# Remplacement des labels selon un dictionnaire de correspondance
sample['prediction'] = sample['prediction'].replace(corresp_dict)

# Calcul de l'accuracy
count_equal = (sample['category'] == sample['prediction']).sum()
total_lines = len(sample)
accuracy_percentage = (count_equal / total_lines) * 100

print(f"Time processing: {processing_time:.4f} seconds")
print(f"Accuracy: {accuracy_percentage:.2f}% ({count_equal}/{total_lines})")

Device set to use cpu
Traitement: 100%|██████████| 100/100 [03:47<00:00,  2.28s/text]

Time processing: 227.8575 seconds
Accuracy: 47.00% (47/100)


SyntaxError: 'return' outside function (3393591218.py, line 37)